In [1]:
#import all libraries
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from scipy import stats
from scipy.stats import norm, skew, skewtest #for some statistics
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from subprocess import check_output
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import mlxtend.regressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf')

ModuleNotFoundError: No module named 'xgboost'

In [62]:
#import the data
train=pd.read_csv("~/TayGang/Ames Housing Data/train.csv")
test=pd.read_csv("~/TayGang/Ames Housing Data/test.csv")
all_data=pd.concat((train,test)).reset_index(drop=True)

#pull out response variable, log it
y_train=all_data.iloc[0:1460,:].SalePrice
y_train=np.log1p(y_train)
all_data=all_data.drop(["SalePrice"],axis=1)

In [2]:
#make a new category for non-missing NA's 
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data["Functional"] = all_data["Functional"].fillna("Typ")

#impute values
#Group by neighborhood where possible and fill in missing value by the median/mode value for that neighborhood
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
all_data['MSZoning'] = all_data.groupby("Neighborhood")['MSZoning'].transform(
    lambda x: x.fillna(x.mode()))
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data.groupby("Neighborhood")['KitchenQual'].transform(
    lambda x: x.fillna(x.mode()))
all_data['Exterior1st'] = all_data.groupby("Neighborhood")['Exterior1st'].transform(
    lambda x: x.fillna(x.mode()))
all_data['Exterior2nd'] = all_data.groupby("Neighborhood")['Exterior2nd'].transform(
    lambda x: x.fillna(x.mode()))
all_data['SaleType'] = all_data.groupby("Neighborhood")['SaleType'].transform(
    lambda x: x.fillna(x.mode()))

#drop Utilities - all but one obs the same
all_data = all_data.drop(['Utilities'], axis=1)

NameError: name 'all_data' is not defined

In [64]:
#hand imputing
all_data.iloc[1460+95,45]=3
all_data.iloc[1915,54]='RM'
all_data.iloc[2216,54]='RM'
all_data.iloc[2250,54]='RM'
all_data.iloc[2904,54]='RL'
all_data.iloc[2151,23]='VinylSd'
all_data.iloc[2151,24]='VinylSd'
all_data.iloc[2489,70]='WD'

In [65]:
#make some continuous variables into categorical variables
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [66]:
#transformations (graphs not included)
all_data["LogLotArea"]=all_data.LotArea.apply(np.log)
all_data["YrBlt2"]=all_data.YearBuilt**2
all_data['TotRmsAbvGrd']=all_data['TotRmsAbvGrd'].astype(int)
all_data['OverallQual']=all_data['OverallQual'].astype(int)
all_data["OverallQ2"]=all_data["OverallQual"]**2

#drop the old one
all_data=all_data.drop(["LotArea","Id"],axis=1)

In [67]:
#some new variabels
all_data['TotalSF']=all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
all_data['TotalBath']=all_data['BsmtFullBath']+.5*all_data['BsmtHalfBath']+all_data['FullBath']+.5*all_data['HalfBath']

#drop the old ones
all_data=all_data.drop(['TotalBsmtSF','1stFlrSF','2ndFlrSF','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath'],axis=1)

In [68]:
#making variables ordinal
all_data=all_data.replace({"Alley" : {"None" : 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"None" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageFinish" : {"None" : 0, "Unf" : 1, "RFn" : 2, "Fin" : 3},
                       "GarageQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"None" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}})

In [69]:
#are there any missing values left?
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio


In [70]:
any(all_data.MSZoning.isna())

False

In [71]:
all_data = pd.get_dummies(all_data)

In [72]:
#define root mean squared error function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(all_data.iloc[0:1460,:].values)
    rmse= np.sqrt(-cross_val_score(model, all_data.iloc[0:1460,:].values, y_train, 
                                   scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [73]:
#run LASSO and XGBOOST
#parameters predefined by Bayesian optimization package
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [74]:
#scores for individual models
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1321 (0.0265)

Xgboost score: 0.1217 (0.0086)



In [75]:
#stack models using linear regression as meta model
stregr = StackingRegressor(regressors=[lasso,model_xgb], 
                           meta_regressor=lr)

In [77]:
stregr.fit(all_data.iloc[0:1460,:], y_train)

StackingRegressor(meta_regressor=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
         regressors=[Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, ran...=0.464,
       reg_lambda=0.8571, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.5213)],
         store_train_meta_features=False, verbose=0)

In [78]:
#predictions are given in logged form, transform back to original scale
meta=np.expm1(stregr.predict(all_data.iloc[1460:,:]))

In [79]:
#create csv for submission
sub=pd.DataFrame()
sub['Id']=test['Id']
sub['SalePrice']=meta
sub.to_csv('stackedsubmission.csv',index=False)